In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import datetime as dt
from plotly import graph_objects as go
import matplotlib.ticker as ticker
import seaborn as sns
from scipy import stats as st
import math as mth
import warnings
warnings.filterwarnings("ignore")
from google.colab import files
uploaded = files.upload()

Saving pgs.csv to pgs.csv


### Предобработка данных

In [4]:
import io
df = pd.read_csv(io.BytesIO(uploaded['pgs.csv']))

In [5]:
df.head()

,Level,ID,Version,Country
0,1,1160211,1,United States
1,2,1160211,1,United States
2,3,1160211,1,United States
3,4,1160211,1,United States
4,5,1160211,1,United States


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62265 entries, 0 to 62264
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Level    62265 non-null  int64 
 1   ID       62265 non-null  int64 
 2   Version  62265 non-null  int64 
 3   Country  60797 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.9+ MB


Имеются пропущенные значения по странам. Возможно при установке игры, пользователь не указал свою страну, чтобы не исключать данные просто заменим пропущенные значения на "Неизвестно".

In [7]:
df['Country']=df['Country'].fillna("Unknown")

In [8]:
df.query('Country=="Unknown"')['ID'].nunique()

313

Проверим наличие дублирующих строк.

In [9]:
df.duplicated().sum()

4981

В данных присутствуют дублирующие строки. Возможно, в выгрузке содержатся данные о нескольких попытках прохождения  одного уровня некоторых пользователей. В целом, для дальнейшего анализа прогресса игроков не будем учитывать данные дублирующие строки, поскольку нам не известна истинная причина их появления. Для дальнейшего анализа результатов  теста мы будем рассматривать конверсию перехода в каждый сегмент, исходя из числа уникальных пользователй. Поэтому избавимся от дублирующих строк.

In [10]:
df=df.drop_duplicates()

In [11]:
df['ID'].nunique()

11775

В тесте участвовало  11 775 пользователей.

Проверим нет ли пересечений между группами.

In [12]:
A=df.query('Version==1')['ID'].unique().tolist()
B=df.query('Version==2')['ID'].unique().tolist()
intersection_set = set.intersection(set(A), set(B))
len(list(intersection_set))

271

Как мы можем заметить, 271 пользователь видел как старую, так и новую версию игры. По правилам проведения АВ-теста пользователи не могут быть одновременно в двух группах, поскольку оценить эффект нововведений становится невозможно. Поэтому исключим из данных пользователей, попавших в две группы.

In [13]:
list1=list(intersection_set)

In [14]:
df=df.query('ID not in @list1')
len(df)

52998

In [15]:
df['ID'].nunique()

11504

В целом после очистки данных мы удалили около 15% наблюдений и исключили примерно 2% пользователей.



### Анализ данных

Посмотрим как пользователи распределились по группам.

In [16]:
df.groupby('Version')['ID'].nunique()

Version
1    4541
2    6963
Name: ID, dtype: int64

Количество пользователей из первой группы превосходит число игроков из второй группы более чем на 35%. В идеале при проведении АB-теста количество участников в группах должно быть равным.

Посмотрим на географию участников теста.

In [17]:
 df['Country'].nunique()

135

In [18]:
df.groupby('Country')['ID'].nunique().sort_values(ascending=False).head(5).reset_index()

,Country,ID
0,Russian Federation,1745
1,United States,790
2,"Iran, Islamic Republic of",790
3,India,739
4,Ukraine,685


Всего в тесте были представлены жители 134 стран (не учитываем неопределенные страны). Больше всего участников теста было из России, на втором месте по числу участников являлись США и Иран, игроки из Индии и Украины закрыли топ-5 стран.

In [19]:
#выгрузим отфильтрованные данные для визуализации некоторых параметров в Tableau
df.to_csv(r'C:\Users\User\Desktop\game\filtered_data.csv', index = False)

Пропишем функцию для выделения сегментов по уровням игры

In [20]:
def step(level):
    if 0<level<=2:
        return "tutorial"
    elif 2<level<=9:
        return "chapter 1"
    elif 9<level<=23:
        return "chapter 2"
    elif 23<level<=43:
        return "chapter 3"
    else: return "chapter 4"

In [21]:
df['step']=df['Level'].apply(step)

In [22]:
А=df.query('Version==1').groupby('step')['ID'].nunique().sort_values(ascending=False).reset_index()
А.columns=['step','users']
А['conversion']=А['users']/4541
А

,step,users,conversion
0,tutorial,4142,0.912134
1,chapter 1,2741,0.603612
2,chapter 2,472,0.103942
3,chapter 3,85,0.018718
4,chapter 4,48,0.010570


In [23]:
B=df.query('Version==2').groupby('step')['ID'].nunique().sort_values(ascending=False).reset_index()
B.columns=['step','users']
B['conversion']=B['users']/6963
B

,step,users,conversion
0,tutorial,6782,0.974005
1,chapter 1,4402,0.632199
2,chapter 2,800,0.114893
3,chapter 3,71,0.010197
4,chapter 4,10,0.001436


Видим, что в обеих группах некоторые пользователи не проходили этап обучения. Возможно они попали в тест, когда у них уже были пройдены данные этапы - навря дли можно сразу попасть на 3 уровень пропустив 1 и 2 . В целом, в дальнейшем при построении воронки прогресса участников исходным числом будем считать число пользователей с этапа обучения.

In [24]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Группа А',
    y = ["tutorial", "ch1", "ch2","ch3", "ch4"],
    x = [4142,2741, 472, 85,48],
    textinfo = "value+percent initial",
    hoverinfo='x+y+percent initial+percent previous'))

fig.add_trace(go.Funnel(
    name = 'Группа В',
    orientation = "h",
    y = ["tutorial", "ch1", "ch2","ch3", "ch4"],
    x = [6782,4402,800,71,10],
    textposition = "inside",
    textinfo = "value+percent initial",
    hoverinfo='x+y+percent initial+percent previous'))
fig.show()

На представленной выше воронке мы видим, что наибольшее число пользователей в обеих группах теряется при переходе на вторую главу. В целом, конверсия в первую и вторую главу почти одинаковая в обеих группах. А вот конверсия прогресса в 3 и 4 главу в 1 версии игры значительно выше в сравнении со второй версией. Проверим статистическую значимость данной разницы.

Для проверки различия конверсий между группами воспользуемся z-критерием, применяемым в случае проверки разности между долями.

**Сформулируем гипотезы:**

* ***H0***: конверсии (в главу 1/ в главу 2/ в главу3/ в главу 4) в версиях 1 и 2 равны
* ***H1*** : конверсии (в главу 1/ в главу 2/ в главу3/ в главу 4) в версиях 1 и 2 различаются

In [28]:
test=df.query('step!="tutorial"').pivot_table(index='step',columns='Version', values='ID', aggfunc='nunique')
test

Version,1,2
step,,
chapter 1,2741,4402
chapter 2,472,800
chapter 3,85,71
chapter 4,48,10


In [29]:
#функция для проведения z-теста
def z_test(g1,g2,step,alpha):
    
    p1=test.loc[step, g1]/4142
    p2=test.loc[step, g2]/6782
    p_comb=(test.loc[step, g1]+test.loc[step, g2])/(4142+6782)
    diff=p1-p2
    
    z=diff/mth.sqrt(p_comb * (1 - p_comb) * (1 /4142 + 1 /6782))
    
    distr = st.norm(0, 1)
    
    p_value = (1 - distr.cdf(abs(z))) * 2
    
    print('Версии игры:  {} и {}\nСегмент: {}\np-value: {p_value:.5f}'.format(g1, g2, step, p_value=p_value))
    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

In [30]:
for step in test.index:
    z_test(1, 2 , step, 0.05)
    print()

Группы:  1 и 2
Сегмент: chapter 1
p-value: 0.17628
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Группы:  1 и 2
Сегмент: chapter 2
p-value: 0.52665
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Группы:  1 и 2
Сегмент: chapter 3
p-value: 0.00002
Отвергаем нулевую гипотезу: между долями есть значимая разница

Группы:  1 и 2
Сегмент: chapter 4
p-value: 0.00000
Отвергаем нулевую гипотезу: между долями есть значимая разница



Согласно проведенным тестам, на пятипроцентном уровне значимости мы можем говорить о том, что конверсии в 1 и 2 главу игры не различаются между версиями игры, а коверсии в 3 и 4 главу имеют статистически значимую разницу: 
* конверсия в 3 главу составляет 2.1% у первой версии игры, а у второй - 1%
* конверсия в 4 главу составляет 1.2% у первой версии игры, а у второй - 0.1%

### Выводы

* В тесте приняли участие 11775 пользователей из 134 стран.
* Наибольшее число участников теста было из России.
* Наибольшее число пользователей в обеих версиях игры теряется при переходе на вторую главу.
* Конверсии в 3 и 4 главы имеют статистически значимую разницу в разных версиях игры - во второй версии конверсия в 3 главу ниже в 2 раза, чем у первой версии, а конверсия в 4 главу во второй версии ниже в 12 раз, чем у первой.
* Таким образом, доработка во второй версии либо усложнила переходы на 3 и 4 главу, или же сделала игру менее привлекательной для пользователей на данных этапах. Если стоит цель удерживать как можно большее количество игроков до последних глав игры, вводить вторую версию игры не стоит.

Часть результатов теста представлена на дашборде https://public.tableau.com/profile/maria7591#!/vizhome/AB-_16206854247920/Dashboard2